# Deploying AI
## Assignment 1: Evaluating Summaries

A key application of LLMs is to summarize documents. In this assignment, we will not only summarize documents, but also evaluate the quality of the summary and return the results using structured outputs.

**Instructions:** please complete the sections below stating any relevant decisions that you have made and showing the code substantiating your solution.

## Select a Document

Please select one out of the following articles:

+ [Managing Oneself, by Peter Druker](https://www.thecompleteleader.org/sites/default/files/imce/Managing%20Oneself_Drucker_HBR.pdf)  (PDF)
+ [The GenAI Divide: State of AI in Business 2025](https://www.artificialintelligence-news.com/wp-content/uploads/2025/08/ai_report_2025.pdf) (PDF)
+ [What is Noise?, by Alex Ross](https://www.newyorker.com/magazine/2024/04/22/what-is-noise) (Web)

# Load Secrets

In [1]:
%load_ext dotenv
%dotenv ../05_src/.secrets

## Load Document

Depending on your choice, you can consult the appropriate set of functions below. Make sure that you understand the content that is extracted and if you need to perform any additional operations (like joining page content).

### PDF

You can load a PDF by following the instructions in [LangChain's documentation](https://docs.langchain.com/oss/python/langchain/knowledge-base#loading-documents). Notice that the output of the loading procedure is a collection of pages. You can join the pages by using the code below.

```python
document_text = ""
for page in docs:
    document_text += page.page_content + "\n"
```

### Web

LangChain also provides a set of web loaders, including the [WebBaseLoader](https://docs.langchain.com/oss/python/integrations/document_loaders/web_base). You can use this function to load web pages.

In [4]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.newyorker.com/magazine/2024/04/22/what-is-noise")

loader.requests_kwargs = {'verify':False}

In [ ]:
document_text = loader.load()

document_text[0] # seeing it

c:\Users\anade\dsi\deploying-ai\deploying-ai-env\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.newyorker.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Document(metadata={'source': 'https://www.newyorker.com/magazine/2024/04/22/what-is-noise', 'title': 'What Is Noise? | The New Yorker', 'description': 'Sometimes we embrace it, sometimes we hate it—and everything depends on who is making it, Alex Ross writes.', 'language': 'en-US'}, page_content='What Is Noise? | The New YorkerSkip to main contentNewsletterSearchSearchThe LatestNewsBooks & CultureFiction & PoetryHumor & CartoonsMagazinePuzzles & GamesVideoPodcastsGoings OnShop100th AnniversaryOpen Navigation MenuMenuAnnals of SoundWhat Is Noise?Sometimes we embrace it, sometimes we hate it—and everything depends on who is making it.By Alex RossApril 15, 2024Noise has come to mean an engulfing barrage of data—less an event than a condition.Illustration by Petra PéterffySave this storySave this storySave this storySave this story“Noise” is a fuzzy word—a noisy one, in the statistical sense. Its meanings run the gamut from the negative to the positive, from the overpowering to the mysteri

## Generation Task

Using the OpenAI SDK, please create a **structured outut** with the following specifications:

+ Use a model that is NOT in the GPT-5 family.
+ Output should be a Pydantic BaseModel object. The fields of the object should be:

    - Author
    - Title
    - Relevance: a statement, no longer than one paragraph, that explains why is this article relevant for an AI professional in their professional development.
    - Summary: a concise and succinct summary no longer than 1000 tokens.
    - Tone: the tone used to produce the summary (see below).
    - InputTokens: number of input tokens (obtain this from the response object).
    - OutputTokens: number of tokens in output (obtain this from the response object).
       
+ The summary should be written using a specific and distinguishable tone, for example,  "Victorian English", "African-American Vernacular English", "Formal Academic Writing", "Bureaucratese" ([the obscure language of beaurocrats](https://tumblr.austinkleon.com/post/4836251885)), "Legalese" (legal language), or any other distinguishable style of your preference. Make sure that the style is something you can identify. 
+ In your implementation please make sure to use the following:

    - Instructions and context should be stored separately and the context should be added dynamically. Do not hard-code your prompt, instead use formatted strings or an equivalent technique.
    - Use the developer (instructions) prompt and the user prompt.


In [17]:
from openai import OpenAI
import os

from langchain.chat_models import init_chat_model

llm = init_chat_model("gpt-4o-mini", 
                      model_provider="openai",
                      base_url='https://k7uffyg03f.execute-api.us-east-1.amazonaws.com/prod/openai/v1',
                      default_headers={"x-api-key": os.getenv('API_GATEWAY_KEY')},
                      )

In [34]:
system_prompt = "You are Sheldon Cooper, from The Big Bang Theory. You hold a PhD in Physics and know that no one is smarter than you. You are very critical with work that is not theoretical physics."

In [35]:
text = document_text[0]

prompt = f"""
    You are the famous physicist Sheldon Cooper. 
    You read the following article and you will provide a summary of the article.
    You also need to mention the name of the article, the author of the article, 
    and why the article is relevant for an AI professional.

    The article is the following: 
    <article>
    {text}
    </article>
"""

In [36]:
from typing import Optional
from pydantic import BaseModel, Field

class Opinion(BaseModel):
    Author: str=Field(description="The author of the article")
    Title: str=Field(description="The title of the article")
    Relevance: str=Field(description="Write a statement, no longer than one paragraph, that explains why is this article relevant for an AI professional in their professional development")
    Summary: str=Field(description="a concise and succinct summary no longer than 1000 tokens, using the tone of the system_prompt."
    )

structured_llm = llm.with_structured_output(Opinion)

output = structured_llm.invoke(prompt)



In [37]:
output

Opinion(Author='Alex Ross', Title='What Is Noise? | The New Yorker', Relevance='This article is particularly relevant for AI professionals as it delves into the nuances of noise, both in a literal and metaphorical sense, paralleling how noise affects data processing and signal transmission in AI systems. Understanding the implications of noise extends to algorithm design and how AI systems handle real-world data, which is often noisy and unstructured.', Summary='In Alex Ross\'s article "What Is Noise?" published in The New Yorker, the multifaceted concept of noise is explored, revealing its complex duality as both a source of frustration and inspiration throughout history and culture. Beginning with its etymological roots linking it to nuisances, the article navigates through the philosophical and sociological interpretations of noise, highlighting its transformative role in music and sound art, while also confronting issues of noise as a manifestation of power. Ross draws connections 

# Evaluate the Summary

Use the DeepEval library to evaluate the **summary** as follows:

+ Summarization Metric:

    - Use the [Summarization metric](https://deepeval.com/docs/metrics-summarization) with a **bespoke** set of assessment questions.
    - Please use, at least, five assessment questions.

+ G-Eval metrics:

    - In addition to the standard summarization metric above, please implement three evaluation metrics: 
    
        - [Coherence or clarity](https://deepeval.com/docs/metrics-llm-evals#coherence)
        - [Tonality](https://deepeval.com/docs/metrics-llm-evals#tonality)
        - [Safety](https://deepeval.com/docs/metrics-llm-evals#safety)

    - For each one of the metrics above, implement five assessment questions.

+ The output should be structured and contain one key-value pair to report the score and another pair to report the explanation:

    - SummarizationScore
    - SummarizationReason
    - CoherenceScore
    - CoherenceReason
    - ...

In [52]:
# This is the original text to be summarized
input = prompt

# This is the summary, replace this with the actual output from your LLM application
actual_output="""
In Alex Ross\'s article "What Is Noise?" published in The New Yorker, the multifaceted concept of noise is explored, revealing its complex duality as both a source of frustration and inspiration throughout history and culture. Beginning with its etymological roots linking it to nuisances, the article navigates through the philosophical and sociological interpretations of noise, highlighting its transformative role in music and sound art, while also confronting issues of noise as a manifestation of power. Ross draws connections between personal experiences of noise intolerance and its broader societal implications, viewing noise not just as unwanted sound but as a crucial element in the human condition—representative of struggle, creativity, and resistance. He discusses various instances of noise across cultures and chronological contexts, including artistic movements and the evolution of sound in urban environments, ultimately proposing that noise, in its chaotic forms, serves as a vehicle for both liberation and control within the complex tapestry of human society.
"""

In [61]:
from deepeval import evaluate
from deepeval.test_case import LLMTestCase
from deepeval.metrics import SummarizationMetric
...

test_case = LLMTestCase(input=input, actual_output=actual_output)
metric = SummarizationMetric(
    threshold=0.5,
    model="gpt-4o-mini",
    assessment_questions=[
        "Does the tone of the output accurately reflect the system_prompt and sounds like Sheldon Cooper?",
        "Are the length of the responses as specified in the instructions?",
        "Is the title of the article correct?", 
        "Is the name of the author of the article correct?",
        "Is the summary formal, including the use of academic jargon?"
    ]
)

# To run metric as a standalone
#metric.measure(test_case)
#print(metric.score, metric.reason)

#evaluate(test_cases=[test_case], metrics=[metric])

In [62]:
# To run metric as a standalone
metric.measure(test_case)
print(metric.score, metric.reason)

Output()

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: any_value. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

# Enhancement

Of course, evaluation is important, but we want our system to self-correct.  

+ Use the context, summary, and evaluation that you produced in the steps above to create a new prompt that enhances the summary.
+ Evaluate the new summary using the same function.
+ Report your results. Did you get a better output? Why? Do you think these controls are enough?

Please, do not forget to add your comments.


# Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

## Submission Parameters

- The Submission Due Date is indicated in the [readme](../README.md#schedule) file.
- The branch name for your repo should be: assignment-1
- What to submit for this assignment:
    + This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
- What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    + Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

## Checklist

+ Created a branch with the correct naming convention.
+ Ensured that the repository is public.
+ Reviewed the PR description guidelines and adhered to them.
+ Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.
